In [48]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [49]:
matrix = pd.read_csv('../../Dropbox/datasets/matrix.csv')

In [50]:
matrix['Total Plays']=matrix.sum(axis=1)
matrix['Total Plays'].describe()

count   24323.00
mean      186.23
std       291.63
min         1.00
25%        23.00
50%        77.00
75%       229.00
max      3992.00
Name: Total Plays, dtype: float64

In [51]:
matrix.head(2)

,user_000001,user_000002,user_000003,user_000004,user_000005,user_000006,user_000007,user_000008,user_000009,user_000010,...,user_000992,user_000993,user_000994,user_000995,user_000996,user_000997,user_000998,user_000999,user_001000,Total Plays
!!!: All My Heroes Are Weirdos,0.00,0.00,0.00,7.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,336.00
!!!: Bend Over Beethoven,0.00,0.00,0.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,196.00


In [52]:
matrix['Total Plays'].quantile(np.arange(.9, 1, .01)),

(0.90    489.00
 0.91    528.00
 0.92    561.00
 0.93    603.00
 0.94    662.68
 0.95    730.00
 0.96    816.00
 0.97    928.00
 0.98   1084.56
 0.99   1401.78
 Name: Total Plays, dtype: float64,)

### Filter Matrix to Include Only Songs that Have Been Played >50 Times

In [53]:
matrix = matrix[matrix['Total Plays'] > 50]
matrix.shape

(14503, 989)

In [54]:
# Summary of Total Plays by Track 
matrix.T['Total Plays'] = matrix.T.sum()

### What Are the Top Played Tracks in This Dataset?

In [55]:
print 'Top Tracks by Total Number of Plays'
matrix['Total Plays'].sort_values(ascending=False).head(20)

Top Tracks by Total Number of Plays


The Postal Service: Such Great Heights                 3992.00
Radiohead: Karma Police                                3534.00
Muse: Supermassive Black Hole                          3483.00
Death Cab For Cutie: Soul Meets Body                   3479.00
The Knife: Heartbeats                                  3156.00
Muse: Starlight                                        3060.00
Arcade Fire: Rebellion (Lies)                          3048.00
Britney Spears: Gimme More                             3004.00
The Killers: When You Were Young                       2998.00
Interpol: Evil                                         2989.00
Kanye West: Love Lockdown                              2950.00
Massive Attack: Teardrop                               2948.00
Death Cab For Cutie: I Will Follow You Into The Dark   2947.00
Muse: Time Is Running Out                              2945.00
Bloc Party: Banquet                                    2906.00
Radiohead: All I Need                                  

### Top 10 Users by Total Plays

In [56]:
matrix.sum().sort_values(ascending=False).head(10)

Total Plays   4336688.00
user_000949     95254.00
user_000791     66321.00
user_000155     47092.00
user_000833     35195.00
user_000298     33438.00
user_000544     32861.00
user_000577     31985.00
user_000543     30222.00
user_000529     29889.00
dtype: float64

### Top 10 Users by Total Unique Plays

In [57]:
matrix.T.astype(bool).sum(axis=1).sort_values(ascending=False).head(10)

Total Plays    14503
user_000691     5828
user_000345     4167
user_000861     4162
user_000577     3920
user_000958     3375
user_000149     3367
user_000774     3348
user_000031     3296
user_000837     3230
dtype: int64

**user_000577 is an interesting 'power user', appearing on both top 10 lists.**

### Convert Matrix to Mean Center Cosine Distance Between Tracks

In [58]:
matrix=matrix.replace(0,np.NaN)

In [59]:
def zero_cent(df, axis=1):
    return df.apply(lambda x: x-x.mean(), axis = axis).replace(np.nan,0)

In [60]:
matrix

,user_000001,user_000002,user_000003,user_000004,user_000005,user_000006,user_000007,user_000008,user_000009,user_000010,...,user_000992,user_000993,user_000994,user_000995,user_000996,user_000997,user_000998,user_000999,user_001000,Total Plays
!!!: All My Heroes Are Weirdos,nan,nan,nan,7.00,nan,nan,nan,nan,nan,nan,...,nan,5.00,nan,nan,nan,nan,nan,nan,nan,336.00
!!!: Bend Over Beethoven,nan,nan,nan,4.00,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,196.00
!!!: Dear Can,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,215.00
!!!: Heart Of Hearts,nan,nan,nan,7.00,nan,nan,nan,nan,nan,nan,...,nan,4.00,nan,nan,nan,nan,nan,nan,nan,638.00
!!!: Must Be The Moon,nan,nan,nan,6.00,nan,nan,nan,nan,nan,nan,...,nan,8.00,nan,nan,nan,nan,nan,nan,nan,362.00
!!!: Myth Takes,nan,nan,nan,3.00,nan,nan,nan,nan,nan,nan,...,nan,6.00,nan,nan,nan,nan,nan,nan,nan,350.00
!!!: Pardon My Freedom,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,247.00
!!!: Sweet Life,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,254.00
!!!: Yadnus,nan,nan,nan,4.00,nan,nan,nan,nan,nan,nan,...,nan,6.00,nan,nan,nan,nan,nan,nan,nan,255.00
"""Weird Al"" Yankovic: Polkarama!",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,87.00


In [61]:
df = zero_cent(matrix)
# cosims={}

In [62]:
df.drop('Total Plays', axis=1, inplace=True)

In [63]:
# cosims={}
# counter = 0
# for i in df.index:
#     counter += 1
#     if counter%100==0:
#         print counter,
#     else:
#         pass
#     cosims[i] = cosine_similarity(df, df.loc[i,:]).ravel()
# cosim_df = pd.DataFrame.from_dict(cosims)
# cosim_df.index = cosim_df.columns

In [64]:
# cosim_df.to_csv('../DSI---Capstone-Project/cosim_df.csv')

In [65]:
cosim_df=pd.read_csv('../DSI---Capstone-Project/cosim_df.csv', index_col='Unnamed: 0')

In [66]:
matrix=matrix.replace(np.NaN,0)

In [25]:
# predicts = []
# for song in matrix.index:
#     index = cosim_df[song].sort_values().iloc[-4:-1].index
#     cosim=cosim_df[song].sort_values().iloc[-4:-1].values
#     predicts.append((cosim[0]*matrix.loc[index[0],'user_000577'] + cosim[1]*matrix.loc[index[1], 'user_000577'])/(cosim[0]+cosim[1]))
# rf = pd.DataFrame(zip(predicts, matrix['user_000577']))
# rf.index = matrix.index
# rf.columns = ['Recommendations', 'Actual Plays']
  


,Recommendations,Actual Plays
!!!: All My Heroes Are Weirdos,0.00,0.00
!!!: Bend Over Beethoven,0.00,0.00
!!!: Dear Can,0.00,0.00
!!!: Heart Of Hearts,1.00,0.00
!!!: Must Be The Moon,0.00,0.00


In [67]:
def generate_recommendations(user, no_songs, new = True):
    predicts = []
    for song in matrix.index:
        index = cosim_df[song].sort_values().iloc[-4:-1].index
        cosim=cosim_df[song].sort_values().iloc[-4:-1].values
        predicts.append((cosim[0]*matrix.loc[index[0],user] + cosim[1]*matrix.loc[index[1], user])/(cosim[0]+cosim[1]))
    rf = pd.DataFrame(zip(predicts, matrix[user]))
    rf.index = matrix.index
    rf.columns = ['Recommendations', 'Actual Plays']
    if new == True:
        return rf[rf['Actual Plays']==0][['Recommendations']].sort('Recommendations', ascending=False).head(no_songs)
    else:
        return rf[['Recommendations']].sort('Recommendations', ascending=False).head(no_songs)


### 'user_000577' Top Plays

In [68]:
matrix[['user_000578']].sort('user_000578', ascending=False).head(60)

,user_000578
Sparklehorse: Shade And Honey,4.00
Hot Snakes: This Mystic Decade,3.00
Bloc Party: Little Thoughts,3.00
Arcade Fire: Une Année Sans Lumière,3.00
Maxïmo Park: Signal And Sign,3.00
Nick Drake: Pink Moon,2.00
Iron & Wine: Woman King,2.00
"Shout Out Louds: Oh, Sweetheart",2.00
Art Brut: Formed A Band,2.00
Iron & Wine: Naked As We Came,2.00


### Generate Top Recommendations for 'user_000577

In [69]:
generate_recommendations('user_000578', 10, new = False)

,Recommendations
Arcade Fire: Crown Of Love,2.49
"The Killers: Andy, You'Re A Star",2.00
Maxïmo Park: Limassol,1.52
The Postal Service: Recycled Air,1.51
The Killers: On Top,1.50
The Postal Service: Clark Gable,1.50
Franz Ferdinand: Come On Home,1.50
Iron & Wine: Naked As We Came,1.49
The Jayhawks: Eyes Of Sarahjane,1.49
Feeder: Pushing The Senses,1.49


### Generate Top New Recommendations for 'user_000577'

In [45]:
generate_recommendations('user_000578', 20, new = True)

,Recommendations
Arcade Fire: Crown Of Love,2.49
"The Killers: Andy, You'Re A Star",2.00
The Killers: On Top,1.50
The Postal Service: Clark Gable,1.50
Franz Ferdinand: Come On Home,1.50
The Jayhawks: Eyes Of Sarahjane,1.49
Feeder: Pushing The Senses,1.49
Maxïmo Park: The Coast Is Always Changing,1.47
The White Stripes: Ball And Biscuit,1.46
Lamb: Zero,1.08


### Create a Function to Evaluate Recommendation Accuracy
* Using a 2/3 of a user's actual listening history to train the recommender, what % of the remaining 1/3 will it recommend?

In [35]:
def evaluate(user):
    predicts = []
    for song in matrix.index:
        index = cosim_df[song].sort_values().iloc[-4:-1].index
        cosim=cosim_df[song].sort_values().iloc[-4:-1].values
        predicts.append((cosim[0]*matrix.loc[index[0],user] + cosim[1]*matrix.loc[index[1], user])/(cosim[0]+cosim[1]))
    
    train = []
    for i in zip([0 for x in range(0, 14503)], matrix[user], matrix[user]):
        train.append(np.random.choice(i, 1)[0])
    matrix['train'] = train
    matrix['predicts'] = predicts
    test_len = matrix[(matrix[user]!=0)&(matrix['train']==0)].shape[0]
    predict_len=float((matrix[(matrix[user]!=0)&(matrix['train']==0)][[user,'train', 'predicts']]\
    .sort('predicts', ascending=False).iloc[0:test_len,:]['predicts'] != 0).sum())
    return predict_len/test_len

In [46]:
evaluate('user_000578')

0.23076923076923078